In [1]:
# --- EVALUATION SCRIPT (DEĞERLENDİRME KODU) ---
# Bu kod, eğitilen modeli test verisi üzerinde dener.

# 1. Gerekli Kütüphaneler
!pip install -q torch transformers peft datasets

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from datasets import load_dataset

# 2. AYARLAR
# Base model (Orjinal hali)
base_model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# Senin eğittiğin model (Hugging Face'deki ismi)
# BURAYI DÜZENLE: Kendi kullanıcı adını yaz
adapter_model_id = "caglademir/Qwen2.5-Coder-1.5B-LoRA-DEEP"

print(f"Test edilecek model: {adapter_model_id}")

# 3. MODELİ YÜKLE
# Önce base modeli yüklüyoruz
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Sonra senin eğittiğin LoRA adaptörünü üzerine ekliyoruz
model = PeftModel.from_pretrained(base_model, adapter_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Modeli değerlendirme moduna al
model.eval()

# 4. TEST VERİSİNİ HAZIRLA
print("Test verisi yükleniyor...")
# Dokümana göre test split kullanılmalı
try:
    dataset = load_dataset("Naholav/CodeGen-Deep-5K", split="test")
except:
    # Eğer test split yoksa train'den küçük bir parça al
    print("Test split bulunamadı, Train setinden örnek alınıyor.")
    dataset = load_dataset("Naholav/CodeGen-Deep-5K", split="train").select(range(5))

# 5. ÖRNEK SORU ÇÖZME FONKSİYONU
def evaluate_model(index=0):
    input_text = dataset[index]['input']
    ground_truth = dataset[index]['solution']

    messages = [
        {"role": "system", "content": "You are an expert Python programmer."},
        {"role": "user", "content": input_text}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    print(f"\n--- SORU {index+1} ---")
    print(input_text[:200] + "...")

    # Model cevap üretiyor
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_new_tokens=256)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Cevabı temizle (Sadece üretilen kısmı al)
    response_code = response.split(prompt)[-1] if prompt in response else response

    print("\n--- MODELİN CEVABI ---")
    print(response_code)
    print("\n--- GERÇEK CEVAP (SOLUTION) ---")
    print(ground_truth[:300] + "...")

# 6. TESTİ ÇALIŞTIR
print("Değerlendirme Başlıyor...")
evaluate_model(0)
evaluate_model(1)

Test edilecek model: caglademir/Qwen2.5-Coder-1.5B-LoRA-DEEP


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

...

CodeGen-Deep-5K.jsonl:   0%|          | 0.00/55.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Test split bulunamadı, Train setinden örnek alınıyor.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Değerlendirme Başlıyor...

--- SORU 1 ---
There are N children in AtCoder Kindergarten. Mr. Evi will arrange the children in a line, then give 1 candy to the first child in the line, 2 candies to the second child, ..., N candies to the N-th c...

--- MODELİN CEVABI ---
system
You are an expert Python programmer.
user
There are N children in AtCoder Kindergarten. Mr. Evi will arrange the children in a line, then give 1 candy to the first child in the line, 2 candies to the second child, ..., N candies to the N-th child. How many candies will be necessary in total?

Constraints

* 1≦N≦100
Input

The input is given from Standard Input in the following format:


N


Output

Print the necessary number of candies in total.

Examples

Input

3


Output

6


Input

10


Output

55


Input

1


Output

1
assistant
def main():
    N = int(input().strip())
    total_candies = (N * (N + 1)) // 2
    print(total_candies)

if __name__ == "__main__":
    main()

--- GERÇEK CEVAP (SOLUTION) ---
def ma